In [71]:
from DeepCodec import *
%matplotlib inline


In [60]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [7]:
N = 512
r = 8
net = DeepCodec(8)
GPU = False
if GPU:
    net.cuda()

In [8]:
transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data',train=True, 
                                        download = True, transform=transform)

Files already downloaded and verified


In [10]:

DC_data = torch.zeros(2*len(trainset),512,1)
for i,data in enumerate(trainset):
    images, labels = data
    gray = rgbtogray(images)
    H,W = gray.size()
    H2 = int(H/2)
    DC_data[2*i] = gray[:H2,:].view(512,1)
    DC_data[2*i+1] = gray[H2:,:].view(512,1)

        

In [11]:
DC_trainset = torch.utils.data.TensorDataset(DC_data,DC_data)
DC_trainloader = torch.utils.data.DataLoader(DC_trainset, batch_size=5,shuffle=True)

In [12]:
train_net(net, DC_trainloader, num_epochs=5000, 
              weightpath='./weights/',save_epoch=250,
              lr=0.001,momentum=0.9)

	[1,  2000] loss: 0.121, 8.028 seconds elapsed


KeyboardInterrupt: 

## Example Reload Net

In [171]:
torch.save(net.state_dict(),'./weights_1.weights')
net2 = DeepCodec(r)
net2.load_state_dict(torch.load('./weights_1.weights'))


In [172]:

for i in range(2):
    img, label = faketrainset[i]
    loss = F.mse_loss(Variable(img),Variable(label))
    print(loss[0])
    rimg = net.forward(Variable(img.unsqueeze(0)))
    print(F.mse_loss(rimg,Variable(label)))
    rimg = net2.forward(Variable(img.unsqueeze(0)))
    print(F.mse_loss(rimg,Variable(label)))
    #plt.figure()
    #plt.imshow()

Variable containing:
 0
[torch.FloatTensor of size 1]

Variable containing:
 1.0126
[torch.FloatTensor of size 1]

Variable containing:
 1.0126
[torch.FloatTensor of size 1]

Variable containing:
 0
[torch.FloatTensor of size 1]

Variable containing:
 1.0329
[torch.FloatTensor of size 1]

Variable containing:
 1.0329
[torch.FloatTensor of size 1]

